<a href="https://colab.research.google.com/github/meetAmarAtGithub/Reva_MachineLearning/blob/main/RandomForest_EnsembleLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import seaborn as sns
import pandas as pd

In [ ]:

my_estimator =[10,100,1000]
my_max_depth = [5,10,15]
my_criterion = ['gini','entropy']
my_min_sample_leaf = [2,5,10,15]
my_min_sample_split = [2,5,10]
scorer = sklearn.metrics.make_scorer(sklearn.metrics.recall_score, average = 'weighted')

In [ ]:
titanic = sns.load_dataset('titanic')
titanic.tail(5)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True


In [ ]:
titanic.drop(['alive','class','embark_town','alone','adult_male','who'], axis=1, inplace=True)

In [ ]:
titanic_age_missing = titanic[titanic['age'].isna()]
titanic_age_missing.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
5,0,3,male,NaN,0,0,8.4583,Q,NaN
17,1,2,male,NaN,0,0,13.0000,S,NaN
19,1,3,female,NaN,0,0,7.2250,C,NaN
26,0,3,male,NaN,0,0,7.2250,C,NaN
28,1,3,female,NaN,0,0,7.8792,Q,NaN


In [ ]:
titanic.iloc[[19,26,28],:]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
19,1,3,female,NaN,0,0,7.2250,C,NaN
26,0,3,male,NaN,0,0,7.2250,C,NaN
28,1,3,female,NaN,0,0,7.8792,Q,NaN


In [ ]:
titanic['age'].median()

28.0

In [ ]:
titanic['age'] = titanic['age'].fillna(titanic['age'].median())
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   survived  891 non-null    int64   
 1   pclass    891 non-null    int64   
 2   sex       891 non-null    object  
 3   age       891 non-null    float64 
 4   sibsp     891 non-null    int64   
 5   parch     891 non-null    int64   
 6   fare      891 non-null    float64 
 7   embarked  889 non-null    object  
 8   deck      203 non-null    category
dtypes: category(1), float64(2), int64(4), object(2)
memory usage: 57.0+ KB


In [ ]:
titanic['embarked']=titanic['embarked'].fillna(titanic['embarked'].mode()[0])

In [ ]:
titanic.deck = titanic.deck.astype('object')

In [ ]:
titanic['deck']= titanic['deck'].fillna('missing')

In [ ]:
titanic = titanic.rename(columns= {"sex":"gender", "sibsp":'siblings', "parch":"parents_child", "Survived":'survived'})

In [ ]:
titanic.head(5)

,survived,pclass,gender,age,siblings,parents_child,fare,embarked,deck
0,0,3,male,22.0,1,0,7.2500,S,missing
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,missing
3,1,1,female,35.0,1,0,53.1000,S,C
4,0,3,male,35.0,0,0,8.0500,S,missing


In [ ]:
for x in titanic.columns:
    if titanic[x].dtype == "object":
        titanic[x] =pd.Categorical(titanic[x]).codes

In [ ]:
titanic.head(5)

,survived,pclass,gender,age,siblings,parents_child,fare,embarked,deck
0,0,3,1,22.0,1,0,7.2500,2,7
1,1,1,0,38.0,1,0,71.2833,0,2
2,1,3,0,26.0,0,0,7.9250,2,7
3,1,1,0,35.0,1,0,53.1000,2,2
4,0,3,1,35.0,0,0,8.0500,2,7


In [ ]:
x = titanic.drop(['survived'],axis=1, inplace=False)
y= titanic['survived']
x.shape

(891, 8)

In [ ]:
print('\n')
y.shape

(891,)

In [ ]:
#Create training and test data
train_x,test_x,train_y,test_y= train_test_split(x,y,test_size=0.2,random_state=999)
train_x.shape
train_y.shape
test_x.shape
test_y.shape

(179,)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest = RandomForestClassifier(class_weight='balanced', random_state=23)

In [ ]:
import time

#start time
start = time.time()

#Building the model
grid_rf = GridSearchCV(estimator=forest, cv=4, scoring=scorer, 
                    param_grid= dict(max_depth = my_max_depth, criterion= my_criterion ,
                                     min_samples_leaf = my_min_sample_leaf, min_samples_split = my_min_sample_split, n_estimators = my_estimator))

grid_rf.fit(train_x, train_y)

end= time.time()

training_time= end-start

print("Time taken to train Random Forest: ", training_time)

Time taken to train Random Forest:  617.9011166095734


In [ ]:
predicted = grid_rf.predict(test_x)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(test_y, predicted))

              precision    recall  f1-score   support

           0       0.81      0.94      0.87       115
           1       0.85      0.61      0.71        64

    accuracy                           0.82       179
   macro avg       0.83      0.77      0.79       179
weighted avg       0.82      0.82      0.81       179



In [ ]:
grid_rf.best_score_

0.8300561797752809

In [ ]:
grid_rf.best_params_

{'criterion': 'entropy',
 'max_depth': 15,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 100}

In [ ]:
grid_rf.best_estimator_.feature_importances_

array([0.08099933, 0.29895451, 0.20127023, 0.04948377, 0.03638988,
       0.22725021, 0.04257789, 0.06307418])